In [1]:
import ast
import datetime
import dateutil.parser
import distutils.dir_util
import glob
import IPython.display
import json
import math
import numpy as np
import os
import pandas as pd
import pathlib
import pickle
import pyautogui
import pytz
import re
import requests
import selenium
import shutil
import sqlite3
import statsapi
import statsmodels.formula.api as smf
import time
import unidecode
import warnings
import webbrowser
import xlrd
import random
import urllib
from urllib.request import urlopen, Request
import zipfile

from bs4 import BeautifulSoup
from copy import deepcopy
from datetime import date
from dateutil import parser
from functools import partial
from IPython.display import display, Javascript
from joblib import Parallel, delayed
from lxml import html
from openpyxl import load_workbook
from pathlib import Path
from pybaseball import statcast
from scipy import stats
from sklearn.preprocessing import StandardScaler
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from statsapi import get
from sqlalchemy import create_engine
from tensorflow import keras
from tqdm import tqdm

from pydfs_lineup_optimizer import get_optimizer, Site, Sport, Player, TeamStack, PlayerFilter, RandomFantasyPointsStrategy
from pydfs_lineup_optimizer.solvers.mip_solver import MIPSolver

import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

# Ensure the warning is ignored only once
warnings.simplefilter(action="ignore")

# Display the DataFrame
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

# Set paths
model_path = r"C:\Users\james\Documents\MLB\Code\Models"
baseball_path = r"C:\Users\james\Documents\MLB\Data2"
download_path = r"C:\Users\james\Downloads"

os.chdir(r"C:\Users\james\Documents\MLB\Code")

In [2]:
# Today's Date
# YYYY-MM-DD (datetime)
todaysdate_dt = datetime.date.today()

# YYYY-MM-DD (string)
todaysdate_dash = str(todaysdate_dt)

# MM/DD/YYYY
todaysdate_slash = todaysdate_dash.split("-")
todaysdate_slash = todaysdate_slash[1] + "/" + todaysdate_slash[2] + "/" + todaysdate_slash[0]

# YYYYMMDD
todaysdate = todaysdate_dash.replace("-", "")

## MM-DD-YYYY
todaysdate_dash = todaysdate[:4] + "-" + todaysdate[4:6] + "-" + todaysdate[6:]

In [3]:
# Get the current date
current_date = datetime.datetime.now()

# Subtract one day from the current date to get yesterday's date
yesterday_dt = current_date - datetime.timedelta(days=1)

# Format yesterday's date as "YYYYMMDD"
yesterdaysdate = yesterday_dt.strftime("%Y%m%d")

# MM/DD/YYYY
yesterdaysdate_slash = yesterdaysdate[4:6] + "/" + yesterdaysdate[6:8] + "/" + yesterdaysdate[0:4] 

## MM-DD-YYYY
yesterdaysdate_dash = yesterdaysdate[:4] + "-" + yesterdaysdate[4:6] + "-" + yesterdaysdate[6:]

In [1]:
# Plate apperance events 
events_list = ['b1','b2','b3','hr','bb','hbp','so','fo','go','lo','po']
# List of statcast stats
statcast_list = ['estimated_woba_using_speedangle','to_left','to_middle','to_right','hard_hit','barrel']
# Stats for which we need to calculate rates
calc_list = ['iso','slg','obp','woba']
# Stats for which we want maximums over time
max_list = ['totalDistance','maxSpeed','maxSpin','launchSpeed']

# Stats for which we want averages over time
avg_list = events_list + statcast_list 

# All stats
stat_list = avg_list + calc_list + max_list + ['ab','pa']

### Batter
# Create position/length-specific inputs from stats
batter_avg_short = [f"{stat}_b" for stat in avg_list]
batter_avg_long =  [f"{stat}_b_long" for stat in avg_list]

batter_calc_short = [f"{stat}_b" for stat in calc_list]
batter_calc_long = [f"{stat}_b_long" for stat in calc_list]

batter_max_short = [f"{stat}_b" for stat in max_list]
batter_max_long =  [f"{stat}_b_long" for stat in max_list]

batter_stats_short = [f"{stat}_b" for stat in stat_list]
batter_stats_long  = [f"{stat}_b_long" for stat in stat_list]

# Steamer stats
batter_stats_fg =     ['b1_rate','b2_rate','b3_rate','hr_rate','bb_rate','hbp_rate','so_rate','woba','slg','obp']


### Pitcher
# Create position/length-specific inputs from stats
pitcher_avg_short = [f"{stat}_p" for stat in avg_list]
pitcher_avg_long =  [f"{stat}_p_long" for stat in avg_list]

pitcher_calc_short = [f"{stat}_p" for stat in calc_list]
pitcher_calc_long = [f"{stat}_p_long" for stat in calc_list]

pitcher_max_short = [f"{stat}_p" for stat in max_list]
pitcher_max_long =  [f"{stat}_p_long" for stat in max_list]

pitcher_stats_short = [f"{stat}_p" for stat in stat_list]
pitcher_stats_long  = [f"{stat}_p_long" for stat in stat_list]

# Steamer stats (for imputing)
pitcher_stats_fg =    ['H9','HR9','K9','BB9','GBrate','FBrate','LDrate','SIERA']
# Add stats for projecting pulls
pitcher_stats_fg2 = pitcher_stats_fg + ['reliability','IP_start']

### Other
# Venues
venue_nums = ['1', '2', '3', '4', '5', '7', '10', '12', '13', '14', '15', '16', '17', '19', '22', '31', '32', 
              '680', '2392', '2394', '2395', '2535', '2536', '2602', '2680', '2681', '2701', '2735', '2756', 
              '2889', '3289', '3309', '3312', '3313', '4169', '4705', '5010', '5325', '5365', '5381', '5445']

venues = [f"venue_{num}" for num in venue_nums]

# Years
years = [f"year_{year}" for year in range(2015,2024)]

# Matchup, weather, and game stat
other_list = ['p_L','b_L','x_vect','y_vect','temperature','onFirst','onSecond','onThird','inning','top','score_diff']
other_list2 = ['x_vect','y_vect','temperature','onFirst','onSecond','onThird','inning','top','score_diff']


### Exclusions
# Stats that do not apply to the position or we just don't want
exclude = ["maxSpeed_b", "maxSpin_b", "maxSpeed_b_long", "maxSpin_b_long", 
           "totalDistance_p", "totalDistance_p_long", "launchSpeed_p", "launchSpeed_p_long",
           "ab_b", "pa_b", "ab_b_long", "pa_b_long", 
           "ab_p", "pa_p", "ab_p_long", "pa_p_long"]

### Input Lists
# Batter inputs (into final mode)
batter_stats = batter_stats_short + batter_stats_long
batter_stats = [item for item in batter_stats if item not in exclude]

# Pitcher inputs (into final mode)
pitcher_stats = pitcher_stats_short + pitcher_stats_long
pitcher_stats = [item for item in pitcher_stats if item not in exclude]

# Hand-specific stats
batter_stats_l = [f'{stat}_l' for stat in batter_stats]
batter_stats_r = [f'{stat}_r' for stat in batter_stats]

pitcher_stats_l = [f'{stat}_l' for stat in pitcher_stats]
pitcher_stats_r = [f'{stat}_r' for stat in pitcher_stats]

# All inputs into final PA model
pa_inputs = batter_stats + pitcher_stats + venues + years + other_list
pa_inputs2 = pa_inputs + ['imp_b', 'imp_p']

# List of rolling stats in pull inputs (these are derived from other status and therefore shouldn't be included in models_inputs_plus)
rolled_sums_list = [f'{stat}_sum' for stat in events_list] + ['rbi_sum', 'faced_sum']

# All inputs into final pulls model (specifically, from the API/Statcast, not Steamer)
pull_inputs = ['inning', 'halfInning', 'awayScore', 'homeScore', 'rbi']
pull_inputs2 = rolled_sums_list + ['pitcher_score', 'batter_score'] + ['IP_start']


# Add additional variables for ease of use
model_inputs_plus = ['date', 'gamePk', 'atBatIndex', 'batterName', 'pitcherName', 'batter', 'pitcher', 'batSide', 'pitchHand', 'eventsModel', 'pa_b', 'ab_b', 'pa_p', 'ab_p'] + pa_inputs2 + pull_inputs + events_list
batter_stats_plus = ['date', 'batterName', 'batter', 'batSide', 'pitchHand'] + batter_stats
pitcher_stats_plus = ['date', 'pitcherName', 'pitcher', 'batSide', 'pitchHand'] + pitcher_stats

# Base running
baserunning_stats = ["sba_2b", "sb_2b", "sba_3b", "sb_3b"]